# Purpose: Apply simulated motion to image and register back using SimpleITK 

In [1]:
import os

In [2]:
import numpy as np
import SimpleITK as sitk

In [3]:
# import serge's libraries 
from compare_transforms import create_slice_timing,construct_transform, view_transform,apply_slicetiming_to_volume

# Load image

In [4]:
# load volume 
rootdir=""
f=rootdir+'Vol0V_sq.nii.gz'
moving_image = sitk.ReadImage(f, sitk.sitkFloat32)

In [5]:
moving_image.GetSize()

(70, 100, 100)

In [6]:
slices=moving_image.GetSize()[0]

In [7]:
savedir="moved/"

In [8]:
os.makedirs(savedir,exist_ok=True)

In [9]:
sliceTiming=create_slice_timing(slices=slices)

# Construct transform

In [17]:
parameters=np.load('tgt_rigid.npy')

In [18]:
parameters

array([-0.24010571,  0.14154703, -0.43887999,  0.04845157,  0.00330419,
       -0.00676707])

In [19]:
ground_truth = sitk.Euler3DTransform()  # For 3D rigid registration

In [23]:
ground_truth=construct_transform([0,0,0,0,0,0]) 

In [24]:
ground_truth.SetParameters(parameters)

In [25]:
view_transform(ground_truth)

Translation in mm  in x,y,z:			 0.05 | 0.0 | -0.01
Rotation    in deg in x,y,z:			 -13.76 | 8.11 | -25.15


In [26]:
# create N transforms that are ALL equal (since slices do no move independently in this example)
transforms=[ground_truth]*len(sliceTiming)

# Simulate motion

In [27]:
fixed_image=apply_slicetiming_to_volume(transforms, sliceTiming, moving_image, interpolator=sitk.sitkLinear, slice_index=0)

In [28]:
# Iterate through the slices and set odd-indexed slices to zero
for z in range(slices):
    if z % 2 != 0:
        # Set the slice to zero
        fixed_image[z, :, :] = 0



In [29]:
sitk.WriteImage(fixed_image, savedir+"slices_moved_v4.nii.gz")

# Register

In [30]:
moving_image = sitk.Cast(moving_image, fixed_image.GetPixelID())


In [31]:
registration_method = sitk.ImageRegistrationMethod()


In [32]:
# Set the similarity metric (mean squares) and interpolator (linear)
registration_method.SetMetricAsMeanSquares()
registration_method.SetInterpolator(sitk.sitkLinear)

In [33]:
# Create a rigid transformation
rigid_transform = sitk.Euler3DTransform()  # For 3D rigid registration

# Set the initial parameters for the rigid transformation
rigid_transform.SetIdentity()
registration_method.SetInitialTransform(rigid_transform)

In [34]:
# Set the transform type (rigid)
registration_method.SetOptimizerAsRegularStepGradientDescent(learningRate=2.0, minStep=1e-4, numberOfIterations=200)

In [35]:
# Perform the registration
final_transform = registration_method.Execute(fixed_image, moving_image)

In [36]:
final_transform.GetParameters()

(-0.2442302871244388,
 0.14660441715027997,
 -0.43915126544800354,
 -0.06284612068891579,
 -0.017519912907873438,
 0.031039574656177005)

In [37]:
# make it viewable - must make it Eulear
final_transform_viewable = sitk.Euler3DTransform()  # For 3D rigid registration
final_transform_viewable.SetParameters(final_transform.GetParameters())

In [38]:
view_transform(final_transform_viewable)

Translation in mm  in x,y,z:			 -0.06 | -0.02 | 0.03
Rotation    in deg in x,y,z:			 -13.99 | 8.4 | -25.16


In [40]:
view_transform(ground_truth)

Translation in mm  in x,y,z:			 0.05 | 0.0 | -0.01
Rotation    in deg in x,y,z:			 -13.76 | 8.11 | -25.15


In [41]:
# Resample the moving image using the final transform
resampled_image = sitk.Resample(fixed_image,moving_image, final_transform.GetInverse(), sitk.sitkLinear, 0.0)

# Save the registered image
sitk.WriteImage(resampled_image, savedir+"slices_moved_v4_back.nii.gz")

In [42]:
# in itksnap 
# itksnap -g Vol0V_sq.nii.gz -o moved/slices_moved_v4.nii.gz moved/slices_moved_v4_back.nii.gz 

## To improve on resampling results 

In [ ]:
# take single slice out of volume and consider sitk image with single slice only (instead of volume) - then resample... 
# this will prevent from black gaps to appear in between linear interp